See: 


https://docs.python.org/3/reference/datamodel.html#emulating-generic-types



neither `__class_getitem__` nor `metaclass` are currently implemented in MicroPython


In [11]:
# python
from typing import Dict, TypeVar

T = TypeVar("T")

DEBUG = True


def trace(func):
    if DEBUG:

        def wrapper(*args, **kwargs):
            print(f"Trace: {func.__qualname__} called with args={args}, kwargs={kwargs}")
            return func(*args, **kwargs)

        return wrapper
    else:
        return func

In [12]:
from __future__ import annotations


class __AC:
    """A class to ignore type hints in code."""

    @trace
    def __init__(*args, **kwargs):
        pass

    @trace
    def __call__(*args, **kwargs):
        # May need some guardrails here
        pass


class __Ignore:
    """A class to ignore type hints in code."""

    @trace
    def __init__(*args, **kwargs):
        pass

    @trace
    def __call__(*args, **kwargs):
        # May need some guardrails here
        pass

    @trace
    def __class_getitem__(cls, arg):
        # fixes: type '__Ignore' is not subscriptable
        return cls

    @trace
    def __getitem__(self, arg):
        # May need some guardrails here
        return __ignore

    @trace
    def __getattr__(self, name):
        if name in self.__dict__:
            return self.__dict__[name]
        return __ignore


__ignore = __Ignore()

Generic = __Ignore

Trace: __Ignore.__init__ called with args=(<__main__.__Ignore object at 0x000002822AB9B090>,), kwargs={}


In [ ]:
# %%micropython

from typing import Final

CONST: Final = 42


print(CONST)

42


In [13]:
class Registry(Generic[T]):
    def __init__(self) -> None:
        self._store: Dict[str, T] = {}

    def set_item(self, k: str, v: T) -> None:
        self._store[k] = v

    def get_item(self, k: str) -> T:
        return self._store[k]

Trace: __Ignore.__class_getitem__ called with args=(<class '__main__.__Ignore'>, ~T), kwargs={}


TypeError: 'classmethod' object is not callable

In [15]:
%load_ext micropython_magic

Current ip.InteractiveTB.mode='Context'
New ip.InteractiveTB.mode='Minimal'


In [20]:
# %%micropython


# Example class without using a metaclass
class MyClass:
    @classmethod
    def __class_getitem__(cls, item):
        return f"Class item: {item}"

    @classmethod
    def __getitem__(cls, item):
        return cls.__class_getitem__(item)


# Usage
print(MyClass["example"])

WARNING  | Traceback (most recent call last):
WARNING  |   File "<stdin>", line 14, in <module>
ERROR    | TypeError: 'type' object isn't subscriptable

WARNING  | b'0.01s - Debugger warning: It seems that frozen modules are being used, which may\r\n'
WARNING  | b'0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off\r\n'
WARNING  | b'0.00s - to python to disable frozen modules.\r\n'
WARNING  | b'0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.\r\n'


MCUException: TypeError: 'type' object isn't subscriptable
